In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")
del stats["Unnamed: 0"]
stats_all_players = stats.fillna(0)
mvp_players = stats_all_players[(stats_all_players["FG%"] > 0.384) & (stats_all_players["MP"] > 25.6) & (stats_all_players["FT%"] > 0.422) & (stats_all_players["PTS"] > 4.7) & (stats_all_players["VORP"] > 1.1)& (stats_all_players["W/L%"] > 0.22)]
mvp_players = stats_all_players

In [3]:
stats_2024 = pd.read_csv("2024_stats.csv")
del stats_2024["Unnamed: 0"]

In [4]:
stats_2024 = stats_2024.fillna(0)
stats_2024 = stats_2024[(stats_2024["FG%"] > 0.384) & (stats_2024["MP"] > 25.6) & (stats_2024["FT%"] > 0.422) & (stats_2024["PTS"] > 4.7) & (stats_2024["VORP"] > 1.1)& (stats_2024["W/L%"] > 0.22)]
condition = (stats_2024['W'] + stats_2024['L']) - stats_2024['G'] < 17
stats_2024 = stats_2024[condition]

In [5]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']

#Predicting on 2024 players stats. Training model on data from previous years (1990 - 2023)
train = mvp_players
test = stats_2024

X_train = train[predictors]
y_train = train["Share"]

X_test = test[predictors]
# y_test = test["Share"]

model = XGBRegressor(learning_rate = 0.05, max_depth = 7, subsample = 0.5)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_XGB = pd.concat([test, predicted_winners], axis = 1)
combinations_XGB = combinations_XGB.sort_values("Predictions", ascending = False)
# combinations = combinations.sort_values("Share", ascending = False)
# combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
# combinations = combinations.sort_values("Predictions", ascending = False)
# combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

In [6]:
combinations_XGB.head(30)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
69,Nikola Jokić,C,28,DEN,79,79,34.6,10.4,17.9,0.583,...,10.6,Denver Nuggets,57,25,0.695,0.0,114.9,109.6,5.23,0.982992
125,Shai Gilgeous-Alexander,PG,25,OKC,75,75,34.0,10.6,19.8,0.535,...,7.1,Oklahoma City Thunder,57,25,0.695,0.0,120.1,112.7,7.36,0.732409
7,Giannis Antetokounmpo,PF,29,MIL,73,73,35.2,11.5,18.8,0.611,...,7.2,Milwaukee Bucks,49,33,0.598,0.0,119.0,116.4,2.44,0.536391
32,Luka Dončić,PG,24,DAL,70,70,37.5,11.5,23.6,0.487,...,8.0,Dallas Mavericks,50,32,0.610,0.0,117.9,115.6,2.30,0.510956
225,Domantas Sabonis,C,27,SAC,82,82,35.7,7.7,13.0,0.594,...,6.2,Sacramento Kings,46,36,0.561,5.0,116.6,114.8,2.29,0.140113
187,Jalen Brunson,PG,27,NYK,77,77,35.4,10.3,21.4,0.479,...,5.4,New York Knicks,50,32,0.610,14.0,112.8,108.2,4.36,0.106743
107,Tyrese Haliburton,PG,23,IND,69,68,32.2,7.2,15.2,0.477,...,5.0,Indiana Pacers,47,35,0.573,2.0,123.3,120.2,2.75,0.073801
327,LeBron James,PF,39,LAL,71,71,35.3,9.6,17.9,0.540,...,5.4,Los Angeles Lakers,47,35,0.573,4.0,118.0,117.4,1.07,0.058983
334,Anthony Edwards,SG,22,MIN,79,78,35.1,9.1,19.7,0.461,...,3.7,Minnesota Timberwolves,56,26,0.683,1.0,113.0,106.5,6.39,0.053156
168,Jayson Tatum,PF,25,BOS,74,74,35.7,9.1,19.3,0.471,...,4.7,Boston Celtics,64,18,0.780,0.0,120.6,109.2,10.75,0.052776


In [24]:
combinations_XGB.head(5).to_csv("XGB_predictions.csv")

In [7]:
model = RandomForestRegressor(bootstrap=True, max_depth=40, max_features = 'sqrt', min_samples_leaf=2, min_samples_split=2, n_estimators = 200)

model.fit(X_train, y_train)
predicted_winners = model.predict(X_test)

predicted_winners = pd.DataFrame(predicted_winners, columns=["Predictions"], index = test.index)
combinations_RF = pd.concat([test, predicted_winners], axis = 1)
combinations_RF = combinations_RF.sort_values("Predictions", ascending = False)

In [8]:
combinations_RF.head(30)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,VORP,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Predictions
69,Nikola Jokić,C,28,DEN,79,79,34.6,10.4,17.9,0.583,...,10.6,Denver Nuggets,57,25,0.695,0.0,114.9,109.6,5.23,0.800363
125,Shai Gilgeous-Alexander,PG,25,OKC,75,75,34.0,10.6,19.8,0.535,...,7.1,Oklahoma City Thunder,57,25,0.695,0.0,120.1,112.7,7.36,0.648019
7,Giannis Antetokounmpo,PF,29,MIL,73,73,35.2,11.5,18.8,0.611,...,7.2,Milwaukee Bucks,49,33,0.598,0.0,119.0,116.4,2.44,0.510470
32,Luka Dončić,PG,24,DAL,70,70,37.5,11.5,23.6,0.487,...,8.0,Dallas Mavericks,50,32,0.610,0.0,117.9,115.6,2.30,0.402103
168,Jayson Tatum,PF,25,BOS,74,74,35.7,9.1,19.3,0.471,...,4.7,Boston Celtics,64,18,0.780,0.0,120.6,109.2,10.75,0.119422
187,Jalen Brunson,PG,27,NYK,77,77,35.4,10.3,21.4,0.479,...,5.4,New York Knicks,50,32,0.610,14.0,112.8,108.2,4.36,0.096226
225,Domantas Sabonis,C,27,SAC,82,82,35.7,7.7,13.0,0.594,...,6.2,Sacramento Kings,46,36,0.561,5.0,116.6,114.8,2.29,0.092307
315,Anthony Davis,C,30,LAL,76,76,35.5,9.4,16.9,0.556,...,4.9,Los Angeles Lakers,47,35,0.573,4.0,118.0,117.4,1.07,0.086927
327,LeBron James,PF,39,LAL,71,71,35.3,9.6,17.9,0.540,...,5.4,Los Angeles Lakers,47,35,0.573,4.0,118.0,117.4,1.07,0.061825
107,Tyrese Haliburton,PG,23,IND,69,68,32.2,7.2,15.2,0.477,...,5.0,Indiana Pacers,47,35,0.573,2.0,123.3,120.2,2.75,0.046005


In [27]:
combinations_RF.head(5).to_csv("RF_predictions.csv")